<a href="https://colab.research.google.com/github/harnalashok/keras/blob/main/pretrained_layers_autoencoder_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 21st Jan, 2021
# Myfolder:
#
# Objectives
#            i)  Building autoencoder using Model class subclassing
#            ii) Training autoencoder with gaussian noise added 
#           iii) Using pre-trained autoencoder layers in a classifier
#           iv)  Comparing Classifer performance with and without pre-trained 
#            v)  Using keras model as a layer
#            vi) A pre-trained model using autoencoder-with-noise added gives
#                better classification
#
#
# Ref: https://www.tensorflow.org/tutorials/generative/autoencoder#first_example_basic_autoencoder
#      https://www.tensorflow.org/tutorials/generative/autoencoder#third_example_anomaly_detection
#      Practical Recommendations for Gradient-Based Training of DeepArchitectures by Yoshua Bengio
#

In [1]:
# 1.0 Import libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [2]:
# 1.1 Display outputs from multiple commands in a colab cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
# 2.0 Get fashion mnist data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Data shape
print (x_train.shape)
print (x_test.shape)


4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


In [4]:
# 2.1 Reshape data for feeding it to NN model
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

In [6]:
# 2.2 Design an Autoencoder with Subclassing
#     Encoder has noise added
#     Ref: https://www.tensorflow.org/guide/keras/custom_layers_and_models
#     Page 313, Book: Hands-on Machine Learning witgh Scitkit-Learn, Keras, and Tensorflow

latent_dim = 64 
class Autoencoder(Model):

  # 2.2.1 Design all layers
  def __init__(self, latent_dim, noise_level=0.1):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.noise_level = noise_level
    # 2.2.2 This is our encoder
    self.encoder = tf.keras.Sequential(
                                        [
                                          layers.Input(shape=(784,)),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.GaussianNoise(0.1),   # Add some noise
                                          layers.Dense(self.latent_dim, activation='relu')
                                        ]
                                       )
    # 2.2.3 This is our decoder
    self.decoder = tf.keras.Sequential(
                                        [
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(784, activation='sigmoid'),
                                          #layers.Reshape((28, 28))
                                         ]
                                       )
  
  # 2.2.4 Call function with just one parameter    
  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded




In [7]:
# 3.0 Instantiate, compile and train autoencoder
autoencoder = Autoencoder(100, 0.1)
autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.fit(x_train, x_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, x_test))


Epoch 1/100
1875/1875 [==============================] - 9s 4ms/step - loss: 0.0388 - val_loss: 0.0176
Epoch 2/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0169 - val_loss: 0.0144
Epoch 3/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0141 - val_loss: 0.0130
Epoch 4/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0128 - val_loss: 0.0120
Epoch 5/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0120 - val_loss: 0.0114
Epoch 6/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0114 - val_loss: 0.0109
Epoch 7/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0109 - val_loss: 0.0107
Epoch 8/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0106 - val_loss: 0.0102
Epoch 9/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0102 - val_loss: 0.0101
Epoch 10/100
1875/1875 [==============================] - 8s 4ms/step - l

In [ ]:
# 3.1 Layer-wise summary
autoencoder.summary()

In [ ]:
# 3.2 Just look at layers
autoencoder.layers 
autoencoder.layers[-2]

In [14]:
# 4.0 Design an Autoencoder with Subclassing
#     BUT Encoder has NO noise added

latent_dim = 64 
class Autoencoder_n(Model):

  # 4.0.1 Design all layers
  def __init__(self, latent_dim, noise_level=0.1):
    super(Autoencoder_n, self).__init__()
    self.latent_dim = latent_dim
    self.noise_level = noise_level
    # 4.0.2 This is our encoder
    self.encoder = tf.keras.Sequential(
                                        [
                                          layers.Input(shape=(784,)),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          #layers.GaussianNoise(0.1),
                                          layers.Dense(self.latent_dim, activation='relu')
                                        ]
                                       )
    # 4.0.3 This is our decoder
    self.decoder = tf.keras.Sequential(
                                        [
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(self.latent_dim, activation='relu'),
                                          layers.Dense(784, activation='sigmoid'),
                                          #layers.Reshape((28, 28))
                                         ]
                                       )
  
  # 4.0.4 Call function with just one parameter    
  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded



In [20]:
# 5.0 As our model has been built using subclassing API,
#     to intantiate the model, we have to fit it.
#     Of course, this training is of no use as we will
#     replace the encoder weights by the learned weights
#     of earlier autoencoder

autoencoder_n = Autoencoder_n(100)
autoencoder_n.compile(optimizer='adam', loss="mse")
autoencoder_n.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))



Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0379 - val_loss: 0.0173
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0164 - val_loss: 0.0142
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0138 - val_loss: 0.0128
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0126 - val_loss: 0.0120
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0118 - val_loss: 0.0115
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0113 - val_loss: 0.0111
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0109 - val_loss: 0.0107
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0105 - val_loss: 0.0105
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0102 - val_loss: 0.0102
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.009

In [21]:
# 5.1 Replace 'encoder' weights of autoencoder_n
autoencoder_n.layers[1].set_weights(autoencoder.layers[1].get_weights())

In [ ]:
# 6.0 So now we have two autoencoders. One which was trained with noise added
#     to input. And the other whose 'encoder' has the same weights as of earlier
#     autoencoder. BUT this autoencoder does NOT have, so-to-say GaussianNoise layer.

# Classification
Using autoencoder pre-trained weights while performing classification

In [22]:
# 7.0 Define Classification model function
#     
def class_model(trainable = False):
  model1 = tf.keras.models.Sequential()
  # 7.1 Add autoencoder_n as a layer
  #     But only the 'encoder' part
  #     WE ADD THAT autoencoder that
  #     DOES NOT have gauusian noise layer
  model1.add(autoencoder_n.layers[-2])
  # 7.2 This is the output layer of our model
  model1.add(layers.Dense(10,activation = "softmax"))
  # 7.3 No training for autoencoder
  autoencoder.layers[-2].trainable = trainable
  model1.layers[0].trainable = trainable
  return model1

In [23]:
# 8.0 Instantiate classification model and train it
model1 = class_model(False)
# 8.1
model1.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")
# 8.2
model1.fit(x_train, y_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, y_test)
                )

Epoch 1/100
1875/1875 [==============================] - 3s 2ms/step - loss: 1.1034 - accuracy: 0.6360 - val_loss: 0.5347 - val_accuracy: 0.8080
Epoch 2/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4997 - accuracy: 0.8224 - val_loss: 0.5013 - val_accuracy: 0.8248
Epoch 3/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4712 - accuracy: 0.8324 - val_loss: 0.4855 - val_accuracy: 0.8288
Epoch 4/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4582 - accuracy: 0.8379 - val_loss: 0.4843 - val_accuracy: 0.8295
Epoch 5/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4483 - accuracy: 0.8398 - val_loss: 0.4857 - val_accuracy: 0.8296
Epoch 6/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4425 - accuracy: 0.8430 - val_loss: 0.4783 - val_accuracy: 0.8296
Epoch 7/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4439 - accuracy: 0.8447 - val_loss: 0.4708 - val_ac

In [24]:
# 8.3 Evaluate model
model1.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.8407


[0.4799702763557434, 0.8406999707221985]

In [25]:
# 8.4 Also get its summary
model1.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_10 (Sequential)   (None, 100)               98700     
_________________________________________________________________
dense_36 (Dense)             (None, 10)                1010      
Total params: 99,710
Trainable params: 1,010
Non-trainable params: 98,700
_________________________________________________________________


In [26]:
# 9.0 Run the classification model again but 
#     this time train the autoencoder layer

model2 = class_model(True)
# 9.1
model2.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")
# 9.2
model2.fit(x_train, y_train,
                epochs=100,
                shuffle=True,
                validation_data=(x_test, y_test)
                )

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6244 - accuracy: 0.7816 - val_loss: 0.4585 - val_accuracy: 0.8394
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3732 - accuracy: 0.8646 - val_loss: 0.4148 - val_accuracy: 0.8595
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3532 - accuracy: 0.8742 - val_loss: 0.4347 - val_accuracy: 0.8615
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3383 - accuracy: 0.8791 - val_loss: 0.4175 - val_accuracy: 0.8649
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3444 - accuracy: 0.8811 - val_loss: 0.4249 - val_accuracy: 0.8651
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3358 - accuracy: 0.8846 - val_loss: 0.5268 - val_accuracy: 0.8440
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3329 - accuracy: 0.8825 - val_loss: 0.5046 - val_ac

In [27]:
# 9.3 Evaluate the model
#    Observe that a pre-trained model using
#    autoencoder gives better classification
model2.evaluate(x_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 2.9957 - accuracy: 0.8628


[2.995737314224243, 0.8628000020980835]

In [28]:
# 10.0 If you evaluate model1 again, we get very low accuracy
#     as autoencoder weights have changed
model1.evaluate(x_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 140.0505 - accuracy: 0.0836


[140.05047607421875, 0.0835999995470047]